In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import random

%pip install gensim
import gensim.downloader
from gensim.models import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity

Note: you may need to restart the kernel to use updated packages.


In [3]:
chosen_event = 'wildfire'

In [44]:
path = f'subgraphs_data/{chosen_event}_subgraph.graphml'
g = nx.read_graphml(path)

The first step is to try to obtain a dictionnary containing the users id and all of their posts in the network as a dictionnary

In [45]:
# let us first get access to the users of the chosen event
users = []
for node, data in g.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':User':
                users.append({node : data})

# now, let us get the tweets related to the chosen event
tweets = []
for node, data in g.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':Tweet':
                tweets.append({node : data})

Select first a sample of 100 users

In [46]:
users
users_sample = random.sample(users, 100)

In [47]:
# build a list containing dictionnaries with the user_node_id and every tweet made by this user
users_posts = []

for u in users_sample:
    user_node_id = [key for key, _ in u.items()][0]
    tweets_by_user = []
    
    for t in tweets:
        tweet_node_id = [key for key, _ in t.items()][0]
        text_tweet = [value for _, value in t.items()][0]['text']

        if tweet_node_id in g[user_node_id]:
            tweets_by_user.append(text_tweet)

    users_posts.append({'user':user_node_id,
                        'tweets':tweets_by_user})

In [9]:
w2v = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [49]:
def sentence_embedding(sentence):
    tokens = sentence.split()
    embeddings = [w2v[token] for token in tokens if token in w2v]
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

In [50]:
users_posts[1]['tweets']

['The @Alberta_UCP didn?t cause the fires in #Alberta but repealing the carbon tax, kneeling to/expanding the oil industry &amp; willfully ignoring and/or denying #climatechange will destroy our environment now &amp; for future generations. Shameful. That?s your ??? #abpoli #abfire #UCP']

In [51]:
for i in users_posts:
    tweets = i['tweets']
    embedded_tweets = []
    for j in i['tweets']:
        emb_tweet = sentence_embedding(j)
        embedded_tweets.append(emb_tweet)
    i['embedded_tweets'] = embedded_tweets

In [71]:
# now, let us compute the user-user similarities with a cosine similarity measure
cos_sim = [[0 for _ in range(100)] for _ in range(100)]

for i in range(len(users_posts)):
    for j in range(len(users_posts)):
        if i == j:
            cos_sim[i][j] = 1
        else:
            emb_i = users_posts[i]['embedded_tweets']
            emb_j = users_posts[j]['embedded_tweets']
            cos_sim[i][j] = cosine_similarity(emb_i, emb_j)[0][0]

In [80]:
np.min(cos_sim)
max_index = np.argmax(cos_sim)
max_row_index, max_col_index = np.unravel_index(max_index, np.array(cos_sim).shape)

In [81]:
max_row_index

8

In [82]:
max_col_index

64